In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('all-MiniLM-L6-v2')
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

docs = [
    "Coral bleaching occurs when corals expel the algae living in their tissues due to stress, often caused by rising sea temperatures.",
    "The Great Barrier Reef is the world's largest coral reef system, located in Australia.",
    "Photosynthesis in coral algae provides most of the energy that corals need to grow.",
    "Climate change is increasing sea temperatures, making coral reefs more vulnerable to bleaching events."
]

doc_embeddings = embedder.encode(docs, convert_to_tensor=True)

/opt/anaconda3/envs/RAGSystems/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/RAGSystems/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [2]:
import torch

def cosine_similarity(vec1, vec2):
    return torch.nn.functional.cosine_similarity(vec1, vec2, dim=0).item()

In [ ]:
# 4. Chat loop
print("RAG Bot: Ask me anything about coral reefs. Type 'exit' to stop.")

while True:
    query = input("\nYou: ")
    if query.lower() == 'exit':
        break

    # Embed query
    query_vec = embedder.encode(query, convert_to_tensor=True)

    # Find most similar doc
    max_sim = -1
    best_idx = -1
    for i, doc_vec in enumerate(doc_embeddings):
        sim = cosine_similarity(query_vec, doc_vec)
        if sim > max_sim:
            max_sim = sim
            best_idx = i

    # Prepare prompt
    context = docs[best_idx]
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"

    # Generate response
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=100)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("\nRAG Bot:", answer.replace(prompt, '').strip())


RAG Bot: Ask me anything about coral reefs. Type 'exit' to stop.



You:  When does coral bleeching occur?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



RAG Bot: Coral bleaching occurs when corals expel the algae living in their tissues due to stress, often caused by rising sea temperatures.
Question: What is the cause of coral bleaching?
Answer: Coral bleaching occurs when corals expel the algae living in their tissues due to stress, often



You:  What provides corals with energy?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



RAG Bot: The algae that grow in coral algae provide most of the energy that corals need to grow.
Question: What does the algae do to the coral?
Answer: The algae that grow in coral algae provide most of the energy that corals need to grow.
Question: What does the algae do to the coral?
Answer:
